In [2]:
using Symbolics
using NLsolve
using LinearAlgebra


In [6]:
# === Define symbols ===
@variables uu ud du dd α p h_e h_p q

# === Define your nonlinear map ===
Fuu = uu*(1 - α*p/2 - (1-α)*(1-p)*(1-h_e)*(1-uu-ud)^q) +
      ud*α*(p/2 + (1-p)*h_p + (1-p)*(1-h_p)*(uu+ud)^q) +
      du*(1-α)*(1 - (1-p)*(1-h_e)*(1-uu-ud)^q)

Fud = uu*α*p/2 +
      ud*((1-α)*(1-p)*(h_e + (1-h_e)*(uu+ud)^q) + α*(1-p/2) -
           α*(1-p)*(h_p + (1-h_p)*(uu+ud)^q)) +
      dd*(1-α)*(1-p)*(h_e + (1-h_e)*(uu+ud)^q)

Fdu = uu*(1-α)*(1-p)*(1-h_e)*(1-uu-ud)^q +
      du*((1-α)*(1-p)*(1-h_e)*(1-uu-ud)^q +
          α*(1 - p/2 - (1-p)*h_p - (1-p)*(1-h_p)*(1-uu-ud)^q)) +
      dd*α*(p/2 + (1-p)*h_p)

Fdd = 1 - (Fuu + Fud + Fdu)

# === Build vector function and compute Jacobian ===
F = [Fuu, Fud, Fdu, Fdd]
vars = [uu, ud, du, dd]

J = Symbolics.jacobian(F, vars)

# === Optional: simplify the Jacobian ===
J_simplified = simplify.(J)

# === Display results ===
println("\nSymbolic 4×4 Jacobian matrix J(uu, ud, du, dd):\n")
display(J_simplified)

# === Example: evaluate numerically ===
subs_dict = Dict(α => 0.1, p => 0.1, h_e => 0.0, h_p => 0.0, q => 3.0,
                 uu => 0.44, ud => 0.06, du => 0.06, dd => 0.44)
J_num = substitute(J_simplified, subs_dict) |> eval
println("\nNumeric Jacobian at example point:\n")
display(J_num)




4×4 Matrix{Num}:
                                                                                                                                                       1 - (1//2)*p*α + (1 - h_e)*(1 - p)*((1 - ud - uu)^q)*(-1 + α) + du*(1 - h_e)*(1 - p)*q*((1 - ud - uu)^(-1 + q))*(1 - α) - (1 - h_e)*(1 - p)*q*uu*((1 - ud - uu)^(-1 + q))*(-1 + α) + (1 - h_p)*(1 - p)*q*ud*((ud + uu)^(-1 + q))*α  …                                                                               0
                                                                                                                                                                                                                    (1//2)*p*α + dd*(1 - h_e)*(1 - p)*q*((ud + uu)^(-1 + q))*(1 - α) + ((1 - h_e)*(1 - p)*q*((ud + uu)^(-1 + q))*(1 - α) - (1 - h_p)*(1 - p)*q*((ud + uu)^(-1 + q))*α)*ud                                    (h_e + (1 - h_e)*((ud + uu)^q))*(1 - p)*(1 - α)
                                                             

4×4 Matrix{Real}:
  1.20155   0.32405   0.79875   0
  0.3047    0.4847    0         0.10125
 -0.19845  -0.2997    0.185     0.005
 -1.3078   -0.50905  -0.98375  -0.10625


Symbolic 4×4 Jacobian matrix J(uu, ud, du, dd):


Numeric Jacobian at example point:



In [7]:
eigenvalues = eigvals(J_num)

# # println("Jacobian matrix:")
# # display(J_matrix)
println("Eigenvalues: ", eigenvalues)
println("Real parts of eigenvalues: ", real.(eigenvalues))
println("Maximum real part: ", maximum(real.(eigenvalues)))

# Stability analysis
if all(abs.(eigenvalues) .< 1)
    println("Fixed point is STABLE")
else
    println("Fixed point is UNSTABLE")
end

Eigenvalues: [2.8099605201941504e-16, 0.07874999999999968, 0.664545142163234, 1.0217048578367693]
Real parts of eigenvalues: [2.8099605201941504e-16, 0.07874999999999968, 0.664545142163234, 1.0217048578367693]
Maximum real part: 1.0217048578367693
Fixed point is UNSTABLE
